In [ ]:
import numpy as np
import os, glob
from skimage import io, exposure, img_as_ubyte
import xtiff

In [ ]:
def write_composite(data_path, plateID, pipeline_name, well, field_num):
    c_filename = os.path.join(data_path, plateID,"Analysis",pipeline_name,"intermediate_files",plateID+"_C_"+well+"_"+str(field_num)+"_reg_stack.ome.tiff")
    print("creating composite movie file for "+c_filename)
    r_src = os.path.join(data_path, plateID,"Analysis/registered_stacks/",plateID+"_R_"+well+"_"+str(field_num)+"_reg_stack.tif")
    g_src = r_src.replace("_R_","_G_")
    p_src = r_src.replace("_R_","_P_")

    cyto_mask_src = os.path.join(data_path, plateID,"Analysis",pipeline_name,"intermediate_files/tracking",well,"field_"+str(field_num),"cyto_filtered_masks")
    cyto_mask_filenames = sorted(glob.glob(cyto_mask_src+"/mask*"))
    nuc_mask_src = os.path.join(data_path, plateID,"Analysis",pipeline_name,"intermediate_files/tracking",well,"field_"+str(field_num),"nuc_filtered_masks")
    nuc_mask_filenames = sorted(glob.glob(nuc_mask_src+"/mask*"))
    r_raw = io.imread(r_src)
    g_raw = io.imread(g_src)
    p_raw = io.imread(p_src)
    r_stack = img_as_ubyte(exposure.rescale_intensity(r_raw, in_range = (np.quantile(r_raw, .07),np.quantile(r_raw, .9995))))
    g_stack = img_as_ubyte(exposure.rescale_intensity(g_raw, in_range = (np.quantile(g_raw, .05),np.quantile(g_raw, .9995))))
    p_stack = img_as_ubyte(exposure.rescale_intensity(p_raw))
    
    cyto_mask_list = []
    for fn in cyto_mask_filenames:
        #read in the mask image
        masks = io.imread(fn)
        cyto_mask_list.append(masks)
    cyto_mask_stack = img_as_ubyte(exposure.rescale_intensity(np.stack(cyto_mask_list)))
    nuc_mask_list = []
    for fn in nuc_mask_filenames:
        #read in the mask image
        masks = io.imread(fn)
        nuc_mask_list.append(masks)
    nuc_mask_stack = img_as_ubyte(exposure.rescale_intensity(np.stack(nuc_mask_list)))
    composite_stack = np.stack((r_stack, g_stack, cyto_mask_stack, nuc_mask_stack, p_stack), axis = 1) #TCCYX
    res = xtiff.to_tiff(composite_stack, c_filename)
    return(composite_stack)


In [ ]:
data_path = "/home/exacloud/gscratch/HeiserLab/images/"
plateID_list = ("HC00701", "HC00801", "HC00901", "HC01001", "HC01301", "HC01401")
plateID_list = (["HC01401"])
plateID_list = ("HC00701", "HC01301")
pipeline_name = "CtcK"
well_list = ("A1", "D2", "D4", "D6")
well_list = (["D6"])

field_num = 1

res = list(map(write_composite, [data_path]*len(well_list)*len(plateID_list), [element for element in plateID_list for i in range(len(well_list))], [pipeline_name]*len(well_list)*len(plateID_list), well_list*len(plateID_list), [field_num]*len(well_list)*len(plateID_list)))